# Do Zero ao Som: Criando Áudio Digital com Python

In [2]:
# Antes de tudo, instale as dependências

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display
import ipywidgets as widgets
from ipywidgets import interactive
from scipy.io import wavfile
from scipy import signal
from abc import ABC, abstractmethod

## Introdução

> *Como o computador processa uma música digitalmente?*
>
> *Como podemos entender uma representação digital de um sinal de áudio?*
>
> *Como sintetizadores digitais são desenvolvidos?*

Neste workshop, vamos entender um pouco sobre o que é um sinal de áudio digital e como podemos criar sons a partir do zero utilizando Python. Tudo isso de maneira interativa e prática.

Divirta-se!

## Som e Sinal

Vamos ganhar uma intuição intuição inicial sobre o que é som e o que é sinal sonoro. Essas definições são importantes para entendermos o que estamos fazendo quando mexemos com áudio digital.

### O que é som?

**Som** é um fenômeno físico que ocorre quando moléculas de ar (ou outro meio elástico) se comprimem e rarefazem rapidamente.
Alto-falantes, por exemplo, movem-se para frente e para trás rapidamente, comprimindo e rarefazendo o ar, criando som.
Por sua vez, nossos ouvidos são capazes de captar essas variações de pressão do ar e transformá-las em sinais elétricos que são enviados ao nosso cérebro.

Podemos modelar esse movimento do ar com a ajuda de computadores e da matemática.

### O que é sinal sonoro?

Um **sinal sonoro** pode ser entendido como uma função matemática que descreve a **variação** de pressão do ar ou tensão elétrica ao longo do tempo.
Podemos visualizar um sinal sonoro com um gráfico onde o eixo horizontal representa o tempo e o eixo vertical representa a amplitude do som em cada instante de tempo.

Caso você nunca tenha visto a onda de uma música antes, observe o sinal dos primeiros 60 segundos da música "*No Ordinary Love*" da cantora Sade no software Ableton Live.

> *This is no ordinary love...*

### Sinais analógicos e digitais

Consegue perceber que temos vários pontos no gráfico da onda sonora? Cada um desses pontos representa a intensidade do som em um instante de tempo.
Por isso, podemos dizer que esse sinal sonoro é **discreto** ou **digital**.

Portanto, podemos imaginar um arquivo de áudio como uma grande lista de números.

In [3]:
sample_rate, audio_file = wavfile.read('Sade-No_Ordinary_Love-60s.wav')

# Mostra o NumPy Dtype
print(audio_file.dtype) # int16, de [-32768, +32767]

# Imprime uma parte do arquivo de áudio
print(audio_file[480000:480000+10])

int16
[[-11313 -14304]
 [-10939 -14153]
 [-11831 -14842]
 [-12245 -15099]
 [-11796 -14719]
 [-11042 -13983]
 [-11059 -13645]
 [-10910 -13119]
 [ -9690 -11869]
 [ -9067 -11190]]


> Dizemos que um sinal sonoro é **analógico** quando a função está definida para todos os instantes no tempo. Ou seja, o sinal sonoro é **contínuo**. Sinais analógicos estão presentes no mundo real, como o som que ouvimos, e também em equipamentos de áudio, como microfones e alto-falantes.
>
> Dizemos que um sinal sonoro é **digital** quando a função está definida apenas para uma lista finita de instantes no tempo. Ou seja, o sinal sonoro é **discreto**. Sinais digitais são utilizados em computadores e equipamentos digitais.

## Amostragem e Quantização

Agora sabemos que um sinal sonoro digital é discretizado no eixo do tempo e também no eixo da amplitude. Mas como isso é feito? Vamos entender um pouco sobre **amostragem** e **quantização**.

> **Amostragem** é o processo de discretização do sinal no eixo do **tempo**.
> Definimos a **taxa de amostragem** como o número de amostras por segundo. A unidade de medida é **Hertz** (Hz). É comum vermos taxas de amostragem de 44.1 kHz, 48 kHz, 96 kHz, etc.

> **Quantização** é o processo de discretização do sinal no eixo da **amplitude**.
> Definimos a **resolução** como o número de níveis de amplitude possíveis. A unidade de medida é **bits**. É comum vermos resoluções de potência de 2, como 8 bits, 16 bits, 24 bits, etc.

Abaixo está um ambiente interativo para você experimentar como uma onda senoidal muda com diferentes frequências, taxas de amostragem e resoluções.

In [4]:
def visualiza_amostragem_e_quantizacao(taxa_amostragem=1000, bits=16, frequencia=1, amplitude=1.0, mostrar_continuo=False):
    # Parâmetros para visualização
    duracao = 1.0
    
    # Sinal contínuo (com mais pontos para parecer contínuo)
    t_continuo = np.linspace(0, duracao, 10000)
    sinal_continuo = np.sin(2 * np.pi * frequencia * t_continuo)
    
    # Amostragem
    t_amostrado = np.linspace(0, duracao, int(taxa_amostragem * duracao))
    onda = np.sin(2 * np.pi * frequencia * t_amostrado)
    
    # Quantização
    resolucao = 2**bits
    onda_quantizada = -1 + 2**(-bits) + 2*np.floor((resolucao-1e-8) * (0.5 + 0.5*onda)) / (resolucao)
    onda_quantizada *= amplitude
    
    # Cria o gráfico
    plt.figure(figsize=(18, 6))
    
    # Mostra o sinal contínuo se o toggle estiver ativado
    if mostrar_continuo:
        plt.plot(t_continuo, sinal_continuo, 'b-', label='Sinal Contínuo', alpha=0.5)
    
    plt.plot(t_amostrado, onda_quantizada, 'r.', label='Sinal Digital')
    
    plt.grid(True)
    plt.xlabel('Tempo (s)')
    plt.ylabel('Amplitude')
    plt.title(f'Senoide ({frequencia} Hz) - {taxa_amostragem} Hz de amostragem, {bits} bits de resolução')
    plt.legend()
    plt.show()
    
    # Reproduz o áudio
    display(Audio(onda_quantizada, rate=taxa_amostragem, normalize=False))

# Cria a interface interativa
interactive_plot = interactive(
    visualiza_amostragem_e_quantizacao,
    taxa_amostragem=widgets.IntSlider(min=1, max=22050, step=10, value=1000),
    bits=widgets.IntSlider(min=1, max=24, step=1, value=16),
    frequencia=widgets.IntSlider(min=1, max=440, step=10, value=1),
    amplitude=widgets.FloatSlider(min=0.1, max=1.0, step=0.1, value=1.0),
    mostrar_continuo=widgets.Checkbox(value=False, description='Mostrar sinal contínuo')
)
display(interactive_plot)

interactive(children=(IntSlider(value=1000, description='taxa_amostragem', max=22050, min=1, step=10), IntSlid…

### Frequência de Nyquist

## Sinais elementares

Vamos estudar alguns sinais elementares que são importantes para entendermos o som: **ruídos** e **sinais periódicos**.

In [5]:
# Vamos definir uma variável global para a taxa de amostragem
taxa_amostragem = 44100

In [6]:
# Vamos definir algumas classes que nos ajudarão 

class AmbienteInterativoSintetizador(ABC):
    def __init__(self, sintetizador):
        self.sintetizador = sintetizador

    def cria_plot(self):
        plt.figure(figsize=(24, 6))
        plt.plot(self.sintetizador.eixo_tempo, self.sintetizador.sinal, 'b-')
        plt.grid(True)
        plt.xlabel('Tempo (s)')
        plt.ylabel('Amplitude')
        plt.ylim(-1.1, 1.1)
        plt.show()
        
    def cria_audio_player(self):
        display(Audio(self.sintetizador.sinal, rate=taxa_amostragem, normalize=False))
    
    @abstractmethod
    def callback():
        pass

In [7]:
class Sintetizador(ABC):
    def __init__(self, amplitude=1.0, duracao=1.0, frequencia=1):
        self.amplitude  = amplitude
        self.duracao    = duracao
        self.frequencia = frequencia
        self.sinal      = self.gerar()
        self.eixo_tempo = None

    @abstractmethod
    def gerar(self):
        pass
    
    @abstractmethod
    def cria_ambiente_interativo(self):
        pass

### Ruídos

Em computação musical, **ruído** não possui uma definição categórica, mas é comum entendermos ele como um sinal sonoro que:

* Não possui uma altura musical definida
* Não costuma ser periódico

Uma caracterização comum de ruídos é por sua **densidade espectral**, ou seja, a distribuição de energia do sinal em função da frequência. Esses ruídos possuem nomes de cores, como **branco**, **vermelho**, **rosa**, etc.

Vamos sintetizar e ouvir os ruídos branco e vermelho.

In [8]:
class AmbienteInterativoRuido(AmbienteInterativoSintetizador):
    def callback(self, amplitude, duracao):
        # Atualiza os parâmetros do sintetizador
        self.sintetizador.amplitude = amplitude
        self.sintetizador.duracao = duracao

        # Gera um novo sinal
        self.sintetizador.sinal = self.sintetizador.gerar()

        self.cria_plot()
        self.cria_audio_player()

In [9]:
class Ruido(Sintetizador):
    def cria_ambiente_interativo(self):
        ambiente = AmbienteInterativoRuido(self)
        interativo = interactive(
            ambiente.callback,
            amplitude=widgets.FloatSlider(min=0, max=1, step=0.01, value=self.amplitude),
            duracao=widgets.FloatSlider(min=0.1, max=1, step=0.1, value=self.duracao),
        )
        display(interativo)

**ATENÇÃO: ABAIXE (MUITO) O VOLUME DO SEU COMPUTADOR ANTES DE OUVIR OS ÁUDIOS.**

#### Ruído branco

O **ruído branco** é um sinal sonoro que possui energia uniformemente distribuída em todas as frequências audíveis.

Talvez você já tenha ouvido ruído branco em um rádio fora de sintonia ou em uma televisão sem sinal. Cuidado, pois esse som pode te incomodar muito!

Vamos sintetizar um ruído branco e ouvi-lo:

In [10]:
class RuidoBranco(Ruido):
    def gerar(self):
        # Gera o eixo do tempo
        self.eixo_tempo = np.linspace(0, self.duracao, int(taxa_amostragem * self.duracao))
        # Gera o sinal de ruído branco
        return np.random.uniform(-self.amplitude, self.amplitude, len(self.eixo_tempo))

In [26]:
ruido_branco = RuidoBranco(amplitude=0.1, duracao=1.0)
ruido_branco.cria_ambiente_interativo()

interactive(children=(FloatSlider(value=0.1, description='amplitude', max=1.0, step=0.01), FloatSlider(value=1…

#### Ruído vermelho

O **ruído vermelho** é um ruído que possui energia inversamente proporcional à frequência. Isso significa que ele possui mais energia em frequências baixas e menos energia em frequências altas.

O ruído vermelho é um ruído mais agradável de se ouvir do que o ruído branco.

Vamos sintetizar um ruído vermelho e ouvi-lo:

In [27]:
class RuidoVermelho(Ruido):
    def gerar(self):
        self.eixo_tempo = np.linspace(0, self.duracao, int(taxa_amostragem * self.duracao))

        amostras = int(taxa_amostragem * self.duracao)

        branco = np.random.uniform(-1, 1, amostras) # O ruído vermelho é derivado do ruído branco

        vermelho = np.zeros(1 * amostras)
        vermelho[0] = branco[0] # Inicializa a primeira amostra

        for i in range(1, amostras):
            vermelho[i] = branco[i] + vermelho[i - 1]

        vermelho = vermelho / np.max(np.abs(vermelho))
        
        vermelho = self.amplitude * vermelho # ajuste de amplitude

        return vermelho

In [29]:
ruido_vermelho = RuidoVermelho(amplitude=1.0, duracao=1.0)
ruido_vermelho.cria_ambiente_interativo() 

interactive(children=(FloatSlider(value=1.0, description='amplitude', max=1.0, step=0.01), FloatSlider(value=1…

#### *Qual a utilidade de ruídos na síntese sonora?*

Podemos concordar que esses ruídos por si só não têm um aspecto musical (ou têm?). Entretanto, a partir de um ruído, podemos modificá-lo (com filtros, envelopes, efeitos, etc.) para criar sons mais complexos e também mais musicais. Vamos explorar isso mais adiante quando falarmos sobre envelopes ADSR.

### Sinais periódicos

Um sinal periódico possui como característica principal a repetição de um padrão de onda ao longo do tempo. Esse padrão pode ser chamado de **ciclo** ou **período**.

A **frequência** de um sinal periódico é o número de ciclos que se repetem em um segundo. Medimos a frequência em **Hertz** (Hz).

A percepção de frequência é o que chamamos de **altura** do som, ou seja, se o som é grave ou agudo.

Os sinais periódicos elementares são as **ondas senoidais**, **ondas quadradas**, **ondas dente-de-serra** e **ondas triangulares**, cada um com um formato de ciclo e características tonais particular.

Vamos sintetizar e ouvir esses sinais periódicos.

In [30]:
class AmbienteInterativoOnda(AmbienteInterativoSintetizador):
    def callback(self, amplitude, duracao, frequencia):
        # Atualiza os parâmetros do sintetizador
        self.sintetizador.amplitude = amplitude
        self.sintetizador.duracao = duracao
        self.sintetizador.frequencia = frequencia
        
        # Gera um novo sinal
        self.sintetizador.sinal = self.sintetizador.gerar() 
        
        # Gera os displays
        self.cria_plot()
        self.cria_audio_player()
        

In [31]:
class Onda(Sintetizador):
    def cria_ambiente_interativo(self):
        ambiente = AmbienteInterativoOnda(self)
        interativo = interactive(
            ambiente.callback,
            amplitude=widgets.FloatSlider(min=0, max=1, step=0.01, value=self.amplitude),
            duracao=widgets.FloatSlider(min=0.1, max=1, step=0.1, value=self.duracao),
            frequencia=widgets.IntSlider(min=1, max=440, step=1, value=self.frequencia)
        )
        display(interativo)

#### Onda senoidal

A **onda senoidal** é um sinal sonoro que possui um único parcial, e por essa causa, é comumente denotada como um "tom puro". Ela é fundamental para a síntese sonora, pois é a base para a construção de muitas outras técnicas de síntese mais sofisticadas.

In [32]:
class OndaSenoidal(Onda):
    def gerar(self):
        # Gera o eixo do tempo
        self.eixo_tempo = np.linspace(0, self.duracao, int(taxa_amostragem * self.duracao))
        # Gera o sinal com a função seno
        return self.amplitude * np.sin(2 * np.pi * self.frequencia * self.eixo_tempo)

In [33]:
onda_senoidal = OndaSenoidal(amplitude=1.0, duracao=1.0, frequencia=1)
onda_senoidal.cria_ambiente_interativo()

interactive(children=(FloatSlider(value=1.0, description='amplitude', max=1.0, step=0.01), FloatSlider(value=1…

Observe o formato da onda senoidal. Não surpreendentemente, ela possui a forma de uma senoide.

Mais importante que isso, ouça o som da onda senoidal. Ela é suave, ao contrário dos ruídos que ouvimos anteriormente.

Tente entender como geramos a onda senoidal. Dê uma olhada na função `gerar` da célula 30.

#### Onda quadrada

A **onda quadrada** é um sinal sonoro 

In [34]:
class OndaQuadrada(Onda):
    def gerar(self):
        self.eixo_tempo = np.linspace(0, self.duracao, int(taxa_amostragem * self.duracao))
        return self.amplitude * signal.square(2 * np.pi * self.frequencia * self.eixo_tempo)

In [35]:
onda_quadrada = OndaQuadrada(amplitude=0.1, duracao=1.0, frequencia=1)
onda_quadrada.cria_ambiente_interativo()

interactive(children=(FloatSlider(value=0.1, description='amplitude', max=1.0, step=0.01), FloatSlider(value=1…

Vamos entender como a onda quadrada é gerada.

#### Onda dente-de-serra

A **onda dente-de-serra** é um sinal sonoro

In [20]:
class OndaDenteDeSerra(Onda):
    def gerar(self):
        self.eixo_tempo = np.linspace(0, self.duracao, int(taxa_amostragem * self.duracao))
        return self.amplitude * signal.sawtooth(2 * np.pi * self.frequencia * self.eixo_tempo)

In [36]:
onda_dente_de_serra = OndaDenteDeSerra(amplitude=0.1, duracao=1.0, frequencia=1)
onda_dente_de_serra.cria_ambiente_interativo()

interactive(children=(FloatSlider(value=0.1, description='amplitude', max=1.0, step=0.01), FloatSlider(value=1…

#### Onda triangular

A **onda triangular** é um sinal sonoro 

In [22]:
class OndaTriangular(Onda):
    def gerar(self):
        self.eixo_tempo = np.linspace(0, self.duracao, int(taxa_amostragem * self.duracao))
        return self.amplitude * signal.sawtooth(2 * np.pi * self.frequencia * self.eixo_tempo, width=0.5)

In [23]:
onda_triangular = OndaTriangular(amplitude=1.0, duracao=1.0, frequencia=1)
onda_triangular.cria_ambiente_interativo()

interactive(children=(FloatSlider(value=1.0, description='amplitude', max=1.0, step=0.01), FloatSlider(value=1…

## ADSR

O som de um instrumento musical não é composto apenas pelo seu caráter harmônico, mas também pela sua **variação dinâmica** ao longo do tempo.

---

*O quanto demora para o som atingir o volume máximo?*

*O quanto demora para o som atingir o volume mínimo?*

*O quanto o som mantém o volume máximo?*

*O quanto demora para o som atingir o volume zero?*

---

Essas perguntas são respondidas pelo conceito de **envelope sonoro**.

Um **envelope sonoro** é uma função que descreve a variação da amplitude de um sinal ao longo do tempo.

Um dos envelopes mais usados no mundo da síntese sonora é o **ADSR**, um acrônimo para **Attack**, **Decay**, **Sustain** e **Release**. Aplicado a um sinal sonoro, o ADSR colabora para a percepção um som como parte de um "instrumento musical".

Vamos ouvir alguns exemplos de ADSR em ação.

> *Hora de abrir o Ableton Live...*

Vamos estudar cada uma das etapas do ADSR:

1. **Attack**: é o tempo que o sinal leva para atingir o seu valor máximo. O ataque é o início do som.
2. **Decay**: é o tempo que o sinal leva para atingir o valor de sustentação.
3. **Sustain**: é o valor que o sinal mantém enquanto a tecla do instrumento é pressionada.
4. **Release**: é o tempo que o sinal leva para atingir o valor zero após a tecla do instrumento ser solta.

Podemos modelar cada uma dessas etapas com funções matemáticas. Para fins didáticos, vamos simplificar essas funções para que sejam funções lineares.

In [37]:
class AmbienteInterativoADSR():
    def __init__(self, adsr):
        self.adsr = adsr

    def cria_plot(self):
        plt.figure(figsize=(18, 6))
        plt.plot(self.adsr.sintetizador.eixo_tempo, self.adsr.sintetizador.sinal, 'b-', label='Sinal Original')
        plt.plot(self.adsr.sintetizador.eixo_tempo, self.adsr.sinal_adsr, 'r-', label='Sinal ADSR')
        plt.grid(True)
        plt.xlabel('Tempo(s)')
        plt.ylabel('Amplitude')
        plt.ylim(-1.1, 1.1)
        plt.legend()
        plt.show()
        
    def cria_audio_player(self):
        display(Audio(self.sintetizador.sinal, rate=taxa_amostragem, normalize=False))
        
    def callback(self,
                 attack, decay, sustain, release,
                 sintetizador,
                 amplitude, duracao, frequencia):
        # Troca de sintetizador
        self.adsr.sintetizador = sintetizador
        
        # Atualiza os parâmetros do sintetizador
        st = self.adsr.sintetizador
        st.amplitude = amplitude
        st.duracao = duracao
        st.frequencia = frequencia

        # Atualiza os parâmetros do ADSR
        adsr = self.adsr
        adsr.attack = attack
        adsr.decay = decay
        adsr.sustain = sustain
        adsr.release = release

        # Gera um novo sinal do sintetizador
        st.eixo_tempo = np.linspace(0, duracao, int(taxa_amostragem * st.duracao))
        st.sinal = st.gerar()
        
        # Gera um novo envelope
        adsr.envelope = adsr.gerar()
        
        # Gera um novo sinal envelopado
        adsr.sinal_adsr = adsr.aplica_envelope()
        
        # Gera o gráfico e reproduz o sinal
        self.cria_plot()
        display(Audio(adsr.sinal_adsr, rate=taxa_amostragem, normalize=False))

class ADSR():
    def __init__(self, a=0.1, d=0.1, s=0.7, r=0.2, sintetizador=onda_senoidal):
        self.attack       = a
        self.decay        = d
        self.sustain      = s
        self.release      = r
        self.sintetizador = sintetizador
        self.envelope     = self.gerar()
        self.sinal_adsr   = self.aplica_envelope()
    
    def gerar(self):
        num_amostras = len(self.sintetizador.eixo_tempo)
        
        # Converte tempos para número de amostras
        a_samples = int(self.attack * taxa_amostragem)
        d_samples = int(self.decay * taxa_amostragem)
        r_samples = int(self.release * taxa_amostragem)
        
        # Calcula o ponto de sustain
        sustain_start = a_samples + d_samples
        sustain_end = num_amostras - r_samples
        
        # Envelope ADSR
        envelope = np.ones(num_amostras)
        
        # Ataque (crescimento linear de 0 a 1)
        if a_samples > 0:
            envelope[:a_samples] = np.linspace(0, 1, a_samples)
        
        # Decay (decrescimento linear de 1 a s)
        if d_samples > 0:
            envelope[a_samples:sustain_start] = np.linspace(1, self.sustain, d_samples)
        
        # Sustain (valor constante s)
        envelope[sustain_start:sustain_end] = self.sustain
        
        # Release (decrescimento linear de s a 0)
        if r_samples > 0:
            envelope[sustain_end:] = np.linspace(self.sustain, 0, r_samples)
            
        return envelope
        
    def aplica_envelope(self):
        return self.sintetizador.sinal * self.envelope
    
    def cria_ambiente_interativo(self):
        ambiente = AmbienteInterativoADSR(self)
        interativo = interactive(
            ambiente.callback,
            attack=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=self.attack, description='Attack (s):'),
            decay=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=self.decay, description='Decay (s):'),
            sustain=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=self.sustain, description='Sustain (0 a 1):'),
            release=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=self.release, description='Release (s):'),
            sintetizador=widgets.Dropdown(
                options={
                    'Ruído Branco': ruido_branco,
                    'Ruído Vermelho': ruido_vermelho,
                    'Senoidal': onda_senoidal,
                    'Quadrada': onda_quadrada,
                    'Dente de Serra': onda_dente_de_serra,
                    'Triangular': onda_triangular
                },
                value=self.sintetizador,
                description='Sintetizador:'
            ),
            amplitude=widgets.FloatSlider(min=0, max=1, step=0.01, value=self.sintetizador.amplitude, description='Amplitude:'),
            duracao=widgets.FloatSlider(min=0.1, max=5.0, step=0.01, value=self.sintetizador.duracao, description='Duração (s):'),
            frequencia=widgets.IntSlider(min=1, max=1000, step=1, value=220, description='Frequência (Hz):')
        )
        display(interativo)

adsr = ADSR()
adsr.cria_ambiente_interativo()

interactive(children=(FloatSlider(value=0.1, description='Attack (s):', max=1.0, step=0.01), FloatSlider(value…

Experimente mexer com todos os parâmetros do ADSR e ouvir o resultado. Você conseguiria simular:

- Um som semelhante a uma tecla de piano sendo pressionada e segurada?
- Um som de um sino de elevador?
- Um som de um sabre de luz?
- Um som de chimbal de bateria acústica?

## Modulação AM

In [38]:
class AmbienteInterativoModulacaoAM:
    def __init__(self, modulacao_am):
        self.modulacao_am = modulacao_am

    def cria_plot(self):
        eixo_tempo = self.modulacao_am.sintetizador.eixo_tempo
        sinal = self.modulacao_am.sintetizador.sinal
        moduladora = self.modulacao_am.moduladora
        sinal_modulado = self.modulacao_am.sinal_modulado

        plt.figure(figsize=(18, 12))
        
        # Plot da portadora
        plt.subplot(311)
        plt.plot(eixo_tempo, sinal, 'b-')
        plt.grid(True)
        plt.xlabel('Tempo (s)')
        plt.ylabel('Amplitude')
        plt.title('Sinal Portadora')
        plt.ylim(-1.1, 1.1)
        
        # Plot da moduladora
        plt.subplot(312)
        plt.plot(eixo_tempo, moduladora, 'r-')
        plt.grid(True)
        plt.xlabel('Tempo (s)')
        plt.ylabel('Amplitude')
        plt.title('Sinal Moduladora')
        plt.ylim(-1.1, 1.1)
        
        # Plot do sinal modulado
        plt.subplot(313)
        plt.plot(eixo_tempo, sinal_modulado, 'g-')
        plt.grid(True)
        plt.xlabel('Tempo (s)')
        plt.ylabel('Amplitude')
        plt.title('Sinal Modulado (AM)')
        plt.ylim(-1.1, 1.1)
        
        plt.tight_layout()
        plt.show()
        
    def cria_audio_player(self):
        display(Audio(self.modulacao_am.sinal_modulado, rate=taxa_amostragem, normalize=False))
        
    def callback(self, freq_moduladora, amp_moduladora, sintetizador, freq_portadora, amp_portadora, duracao):
        # Troca de sintetizador
        self.modulacao_am.sintetizador = sintetizador
        
        # Atualiza os parâmetros do sintetizador
        st = self.modulacao_am.sintetizador
        st.amplitude = amp_portadora
        st.frequencia = freq_portadora
        st.duracao = duracao
        
        # Atualiza os parâmetros da moduladora
        self.modulacao_am.freq_moduladora = freq_moduladora
        self.modulacao_am.amp_moduladora = amp_moduladora
        
        # Gera um novo sinal do sintetizador
        st.eixo_tempo = np.linspace(0, duracao, int(taxa_amostragem * st.duracao))
        st.sinal = st.gerar()
        
        # Gera um novo sinal modulador
        self.modulacao_am.moduladora = self.modulacao_am.gerar()
        
        # Gera um novo sinal modulado
        self.modulacao_am.sinal_modulado = self.modulacao_am.aplica_modulacao()
        
        # Mostra o gráfico e reproduz o áudio
        self.cria_plot()
        self.cria_audio_player()

class ModulacaoAM:
    def __init__(self, sintetizador=onda_senoidal, freq_portadora=220, amp_portadora=1.0, 
                 freq_moduladora=5, amp_moduladora=0.5, duracao=1.0):
        self.freq_moduladora         = freq_moduladora
        self.amp_moduladora          = amp_moduladora
        self.sintetizador            = sintetizador
        self.sintetizador.frequencia = freq_portadora
        self.sintetizador.amplitude  = amp_portadora
        self.duracao                 = duracao
        self.moduladora              = self.gerar()
        self.sinal_modulado          = self.aplica_modulacao()
    
    def gerar(self):
        return self.amp_moduladora * np.sin(2 * np.pi * self.freq_moduladora * self.sintetizador.eixo_tempo)
    
    def aplica_modulacao(self):
        return self.sintetizador.sinal * self.moduladora
    
    def cria_ambiente_interativo(self):
        ambiente = AmbienteInterativoModulacaoAM(self)
        interativo = interactive(
            ambiente.callback,
            freq_moduladora=widgets.IntSlider(min=1, max=100, step=1, value=self.freq_moduladora, description='Freq. Moduladora:'),
            amp_moduladora=widgets.FloatSlider(min=0.1, max=1.0, step=0.1, value=self.amp_moduladora, description='Amp. Moduladora:'),
            sintetizador=widgets.Dropdown(
                options={
                    'Ruído Branco': ruido_branco,
                    'Ruído Vermelho': ruido_vermelho,
                    'Senoidal': onda_senoidal,
                    'Quadrada': onda_quadrada,
                    'Dente de Serra': onda_dente_de_serra,
                    'Triangular': onda_triangular
                },
                value=self.sintetizador,
                description='Portador:'
            ),
            freq_portadora=widgets.IntSlider(min=50, max=1000, step=10, value=self.sintetizador.frequencia, description='Freq. Portadora:'),
            amp_portadora=widgets.FloatSlider(min=0.1, max=1.0, step=0.1, value=self.sintetizador.amplitude, description='Amp. Portadora:'),
            duracao=widgets.FloatSlider(min=0.1, max=5.0, step=0.1, value=self.duracao, description='Duração (s):')
        )
        display(interativo)

am = ModulacaoAM()
am.cria_ambiente_interativo()

interactive(children=(IntSlider(value=5, description='Freq. Moduladora:', min=1), FloatSlider(value=0.5, descr…